In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.cluster import KMeans, k_means_, DBSCAN, AgglomerativeClustering
from sklearn import cluster
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_distances

import hdbscan
from gensim import models
from gensim.corpora import mmcorpus, Dictionary
from gensim.models import lsimodel, ldamodel, tfidfmodel, rpmodel, logentropy_model, TfidfModel, LsiModel, LdaModel
from gensim import matutils, corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models import doc2vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC
import sys
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from konlpy.utils import pprint
mecab = Mecab()
%matplotlib inline

In [2]:
dataDict = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
print (len(dataDict))

keywordsDict = pickle.load(open('./data/pre_data/keywords/keywords_Naver.pickled','rb'))

for idx in dataDict:
    dataDict[idx]['extracted_keywords'] = keywordsDict[idx]

df = pd.DataFrame.from_dict(dataDict, orient='index')
df['date'] = pd.to_datetime(df['date']).dt.date
df.reset_index(inplace = True)
df.rename(columns={'index':'id'}, inplace=True)
print (df.shape)
df.head()

7681
(7681, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a29c445588c132954d1973a,정치,2017-12-07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{미국, 고위, 도화선, 조선반도, 북한, 핵전쟁, 중앙, 대변인}"
1,5a29c445588c132954d1973b,정치,2017-12-07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{의원, 예산안, 정부안, 호남, 지역구, 국민의당, 증액}"
2,5a29c445588c132954d1973c,정치,2017-12-07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{의원, 검찰, 정기국회, 구속영장 청구, 혐의, 국정원장, 조사}"
3,5a29c445588c132954d1973d,정치,2017-12-07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{감사원장, 생활, 지명, 후보자, 법관, 공직 사회}"
4,5a29c445588c132954d1973e,정치,2017-12-07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{김정은, 공장, 양강도, 사무차장, 접경, 펠트먼, 시찰, 삼지연, 훈련}"


In [3]:
def nav_tokenizer(corpus):
    pos = mecab.pos(corpus)
    res = [x[0] for x in pos if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    return res

In [4]:
def remove_stopwords(words, stopwords):
    res = [x for x in words if x not in stopwords]
    res = [x for x in words if len(x) != 1]
    return res

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [6]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [7]:
w2v_docs = list()
for idx in tqdm(df.index):
    text = df.loc[idx,'title']+'.\n'+df.loc[idx,'mainText']
    pos = nav_tokenizer(text)
    pos = remove_stopwords(pos, stopwords)
    label = df.loc[idx, 'category']
    w2v_docs.append(TaggedDocument(pos, label))

100%|██████████| 7681/7681 [01:47<00:00, 71.33it/s]


In [8]:
train, test = train_test_split(w2v_docs, test_size = 0.15)

In [33]:
pickle.dump(train,open('./data/pre_data/train_test_Data/pre_data_word2vec_train_for_keywords_classification_by_mecab.pickled','wb'))
pickle.dump(test,open('./data/pre_data/train_test_Data/pre_data_word2vec_test_for_keywords_classification_by_mecab.pickled','wb'))

In [ ]:
train = pickle.load(open('./data/pre_data/train_test_Data/pre_data_word2vec_train_for_keywords_classification_by_mecab.pickled','rb'))
test = pickle.load(open('./data/pre_data/train_test_Data/pre_data_word2vec_test_for_keywords_classification_by_mecab.pickled','rb'))

In [9]:
x_train = [ x.words for x in train] 
y_train = [ x.tags for x in train] 
x_test = [ x.words for x in test] 
y_test = [ x.tags for x in test] 

## word2vec

In [10]:
w2v_model = Word2Vec(sg=1, size=3000,window=8, workers = 6, iter = 20)
w2v_model.build_vocab(tqdm(x_train))
w2v_model.train(tqdm(x_train), total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

100%|██████████| 6528/6528 [02:09<00:00, 50.58it/s] 


30253911

In [11]:
w2v_model.save('./model/keywords_classification_naver_by_word2vec_size3000_window8_iter20_by_mecab.model')

In [ ]:
w2v_model = Word2Vec.load('./model/keywords_classification_naver_by_word2vec_size3000_window8_iter20_by_mecab.model')

In [12]:
print(w2v_model.most_similar('신혜선'))

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('박시후', 0.7202842235565186), ('서지안', 0.6481706500053406), ('유인영', 0.6313196420669556), ('이태환', 0.6226440668106079), ('선우혁', 0.5971068143844604), ('해성그룹', 0.5866956114768982), ('서은수', 0.5836597681045532), ('소현경', 0.5730416178703308), ('황금빛', 0.5674773454666138), ('도경', 0.559171199798584)]


In [13]:
print(w2v_model.most_similar('손흥민'))

[('토트넘', 0.6285474300384521), ('라멜라', 0.5637016296386719), ('요렌테', 0.5609951019287109), ('스토크시티', 0.5544713139533997), ('에릭센', 0.5456897020339966), ('델레', 0.5351120233535767), ('군면제', 0.5298911333084106), ('원맨쇼', 0.5297625064849854), ('아포엘', 0.5265060663223267), ('웸블리', 0.5245385766029358)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [14]:
print(w2v_model.most_similar('안철수'))

[('국민의당', 0.5822474360466003), ('반안', 0.561185896396637), ('좌고우면', 0.5415193438529968), ('박지원', 0.5413402915000916), ('재신임', 0.5281667709350586), ('용팔이', 0.5226794481277466), ('선관위원장', 0.5172231197357178), ('합당', 0.517013430595398), ('김동호', 0.5131103992462158), ('간신배', 0.5127516984939575)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [15]:
print(w2v_model.most_similar('이명박'))

[('조갑제', 0.43749940395355225), ('사이버사령부', 0.42926695942878723), ('박근혜', 0.4245365858078003), ('실소유주', 0.42140993475914), ('원전수주', 0.42098990082740784), ('동영', 0.41509518027305603), ('박근혜정부', 0.41487938165664673), ('개표기', 0.4140171706676483), ('박연차', 0.4110628366470337), ('국교단절', 0.4107666313648224)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [16]:
print(w2v_model.most_similar('문재인'))

[('배재만', 0.43066489696502686), ('문재', 0.42593008279800415), ('서미선', 0.4231913089752197), ('대통령', 0.4166738986968994), ('수석보좌관', 0.39960482716560364), ('케어', 0.3994876444339752), ('영빈관', 0.38886505365371704), ('효동', 0.3874832093715668), ('의료수가', 0.37777698040008545), ('박지환', 0.3772933781147003)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [17]:
print(w2v_model.most_similar('박근혜'))

[('친박', 0.45818763971328735), ('현기환', 0.42886149883270264), ('이보배', 0.4250842332839966), ('이명박', 0.4245366156101227), ('비선실세', 0.4182092547416687), ('정호성', 0.4149579405784607), ('이원종', 0.4132307767868042), ('국정농단', 0.41193023324012756), ('고동욱', 0.41019147634506226), ('홍용표', 0.4100880026817322)]


/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform(x_train)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 10934


In [19]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [20]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
6528it [00:36, 178.66it/s]
1153it [00:05, 192.26it/s]


In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [22]:
le.fit(y_train)
le.classes_

array(['IT/과학', '경제', '사회', '생활/문화', '세계', '스포츠', '연예', '정치'],
      dtype='<U5')

In [23]:
from keras.utils import np_utils
y_test2 = np_utils.to_categorical(le.transform(y_test),8)
y_train2 = np_utils.to_categorical(le.transform(y_train),8)

Using TensorFlow backend.
/Users/hyunyoun/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [24]:
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.layers import Input, Dense, Embedding, embeddings, merge, Dropout
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=3000))
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(8, activation='sigmoid'))
model.compile(#optimizer='rmsprop',
    optimizer='adadelta',
              #loss='binary_crossentropy',
    #optimizer=SGD(lr=0.2), 
    loss='categorical_crossentropy',
    metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train2, epochs=100, verbose=0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model.evaluate(test_vecs_w2v, y_test2, verbose=2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


0.91066782307


In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
model2 = Sequential()
model2.add(Dense(64, activation='relu', input_dim=3000))
model2.add(Dropout(0.25))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.125))
model2.add(Dense(8, activation='softmax'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
model2.fit(train_vecs_w2v, y_train2,
          epochs=100, verbose=0,
          callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
score = model2.evaluate(test_vecs_w2v, y_test2)
print (score[1])

HBox(children=(IntProgress(value=0, description='Training'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=6528), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 96', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 97', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 98', max=6528), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 99', max=6528), HTML(value='')))


1153/1153 [==============================] - 0s 142us/step
0.914137033825


In [26]:
dataDict2 = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
print (len(dataDict2))

keywordsDict2 = pickle.load(open('./data/pre_data/keywords/keywords_daum.pickled','rb'))

for idx in dataDict2:
    dataDict2[idx]['extracted_keywords'] = keywordsDict2[idx]

df2 = pd.DataFrame.from_dict(dataDict2, orient='index')
df2['date'] = pd.to_datetime(df2['date']).dt.date
df2.reset_index(inplace = True)
df2.rename(columns={'index':'id'}, inplace=True)
print (df2.shape)
df2.head()

4722
(4722, 11)


,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017-12-07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{세보 효과, 연기, 건강, 부작용, 공장, 지역, 남성들}"
1,5a2a61bf588c13481c229d1f,뉴스,2017-12-07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{철분, 치아, 건강, 자세, 디스크, 커피, 낮잠, 점심 식사, 입냄새, 식후}"
2,5a2a61bf588c13481c229d20,뉴스,2017-12-07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{주변, 앙심, 아들, 진술, 범행, 지인, 철원, 경찰, 남편, 성관계}"
3,5a2a61bf588c13481c229d21,뉴스,2017-12-07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{신영자 이사장, 검찰, 유통업체, 네이처리퍼블릭, 매장, 혐의, 롯데, 징역}"
4,5a2a61bf588c13481c229d22,뉴스,2017-12-07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{폭행, 유치장, 범행, 과자, 혐의, 쪽지, 경찰, 남자친구}"


In [27]:
t1 = df2.title.values+'\.n'+df2.mainText.values
da = list()
for t in tqdm(t1):  
    x1 = nav_tokenizer(t)
    x1 = remove_stopwords(x1, stopwords)
    da.append(x1)

100%|██████████| 4722/4722 [01:21<00:00, 58.15it/s]


In [28]:
daum_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, da))])
daum_w2v = scale(daum_w2v)

0it [00:00, ?it/s]/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
4722it [00:28, 163.60it/s]


In [29]:
model_pre = model.predict_classes(daum_w2v)
model2_pre = model2.predict_classes(daum_w2v)

In [30]:
mp1 = le.inverse_transform(model_pre)
mp1

array(['세계', '생활/문화', '사회', ..., '스포츠', '스포츠', '스포츠'],
      dtype='<U5')

In [31]:
mp2 = le.inverse_transform(model2_pre)
mp1

array(['세계', '생활/문화', '사회', ..., '스포츠', '스포츠', '스포츠'],
      dtype='<U5')

In [32]:
(mp1==mp2).sum() / len(daum_w2v)

0.90914866581956799

In [49]:
embedding_matrix.shape

(24206, 3000)

In [34]:
embedding_matrix = np.zeros((len(w2v_model.wv.vocab), 3000))
for i in range(len(w2v_model.wv.vocab)):
    embedding_vector = w2v_model.wv[w2v_model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [51]:
valid_size = 50  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# input words - in this case we do sample by sample evaluations of the similarity
valid_word = Input((1,), dtype='int32')
other_word = Input((1,), dtype='int32')
# setup the embedding layer
embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1],
                      weights=[embedding_matrix])
embedded_a = embeddings(valid_word)
embedded_b = embeddings(other_word)
similarity = merge([embedded_a, embedded_b], mode='cos', dot_axes=2)
# create the Keras model
k_model = Model(input=[valid_word, other_word], output=similarity)

def get_sim(valid_word_idx, vocab_size):
    sim = np.zeros((vocab_size,))
    in_arr1 = np.zeros((1,))
    in_arr2 = np.zeros((1,))
    in_arr1[0,] = valid_word_idx
    for i in range(vocab_size):
        in_arr2[0,] = i
        out = k_model.predict_on_batch([in_arr1, in_arr2])
        sim[i] = out
    return sim

# now run the model and get the closest words to the valid examples
for i in range(valid_size):
    valid_word = w2v_model.wv.index2word[valid_examples[i]]
    top_k = 8  # number of nearest neighbors
    sim = get_sim(valid_examples[i], len(w2v_model.wv.vocab))
    nearest = (-sim).argsort()[1:top_k + 1]
    log_str = 'Nearest to %s:' % valid_word
    for k in range(top_k):
        close_word = w2v_model.wv.index2word[nearest[k]]
        log_str = '%s %s,' % (log_str, close_word)
    print(log_str)

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("me...)`
  


Nearest to 전망: 견조, 증권가, 목표주가, 시노, 의사록, 슈퍼사이클, 전망치, 증권업계,
Nearest to 필요: 수탁자, 사용후핵연료, 의료수가, 앙전, 한국금융연구원, 입법조사처, 조력자, 선불,
Nearest to 사진: 동취, 귀욤, 디어클라우드, 방송화면, 독수리, 자태, 조남기, 김가연,
Nearest to 사용: 치실, 티타늄, 대체기술, 새것, 공유재산, 안드로이드용, 웹표준, 부팅,
Nearest to 애플: 아이폰, 아이폰6, 아이폰7, 팀쿡, 운영체제, 줄소송, 아이폰5, 꺼짐,
Nearest to 예상: 강원산, 경기동부, 전망치, 서남부, 충청권, 경북지역, 내다보, 서해5도,
Nearest to 이번: 공동기자회견, 복병, 중하위권, 불용, 김철근, 수석비서관, 리얼미터, 라주,
Nearest to 지역: 전라, 서남부, 영서지방, 충청도, 양구, 충청권, 북서, 경기동부,
Nearest to 예정: 정병근, 선샤인, 첫방송, 규제개혁위원회, 분관, 스마트공항, 귀국길, 로킹,
Nearest to 가격: 온스, 폭락, 폭등세, 출렁이, 널뛰, 설렁탕, 신선설농탕, 고점,
Nearest to 지난해: 전세자금대출, 전년도, 와츠, 민영보험, 내국인, 상품수지, 누계, 출고량,
Nearest to 오후: 상행선, 기흥구, 분경, 그랜드인터컨티넨탈, 4시, 대회의실, 정오, 고매동,
Nearest to 결과: 선거인단, 선관위원장, 표본오차, 오메가-3, 문진, 세균감염, 양경무, 영역별,
Nearest to 기자: 카드뉴스, 박상, 이재은, 김민정, 그랜드인터컨티넨탈, 류석우, 김수완, 조원진,
Nearest to 조사: 표본오차, 수간호사, 설문, 동영, 참고인, 건과, 양경무, 현준,
Nearest to 트럼프: 도널드, 행정부, 백악관, 재러드, 슈너, 로켓맨, 대통령, 하누카,
Nearest to 오전: 상행선, 영흥면, 시화병원, 인왕산, 한종, 김기훈, 남서방, 창동,
Nearest to 가운데: 최고층, 경기동부